In [29]:
# Compare the modification dates in Databank and DDH for discrepancies
import numpy as np
import wbgapi as wb
import ddh
import sys
from datetime import datetime
# from tabulate import tabulate
# from IPython.display import HTML
import pandas as pd

ddh.load('datacatalog.worldbank.org')

In [30]:
headers = ['NID', 'DDH_TITLE', 'DDH_MODIFIED', 'DBANK_ID', 'DBANK_TITLE', 'DBANK_MODIFIED', 'DISCREPANCY']
# rows = []
df = pd.DataFrame(columns=headers)

for k,v in ddh.dataset.search(['field_wbddh_modified_date', 'field_ddh_harvest_sys_id'], {'field_ddh_harvest_src': 'Indicators API'}):
    database_id = v['field_ddh_harvest_sys_id']['und'][0]['value']
    mod_date    = v['field_wbddh_modified_date']['und'][0]['value'] if v['field_wbddh_modified_date'] else ''
    mod_date = mod_date.split(' ')[0]
    dbank = wb.source.get(database_id)
    delta = np.nan
    try:
        date1 = datetime.strptime(dbank['lastupdated'], '%Y-%m-%d')
        date2 = datetime.strptime(mod_date, '%Y-%m-%d')
        delta = (date2-date1).days
    except:
        pass

    row = [k, v['title'], mod_date, database_id, dbank['name'], dbank['lastupdated'], delta]
    # rows.append(row)
    df.loc[len(df)] = row

# HTML(tabulate(rows, tablefmt='html', headers=['NID','DDH_TITLE', 'DDH_MODIFIED', 'DBANK_ID', 'DBANK_TITLE', 'DBANK_MODIFIED', 'DISCREPANCY']))

In [31]:
df[df['DISCREPANCY']!=0]

,NID,DDH_TITLE,DDH_MODIFIED,DBANK_ID,DBANK_TITLE,DBANK_MODIFIED,DISCREPANCY
10,94622,Quarterly Public Sector Debt,2019-12-03,20,Quarterly Public Sector Debt,2019-12-31,-28
12,94554,Doing Business,2019-11-22,1,Doing Business,2019-10-23,30
31,94605,Millennium Development Goals,2017-12-19,19,Millennium Development Goals,2018-09-19,-274
32,94564,Enterprise Surveys,2017-01-30,13,Enterprise Surveys,2017-01-03,27
35,94623,Readiness for Investment in Sustainable Energy,2014-11-30,44,Readiness for Investment in Sustainable Energy,2015-06-11,-193
37,94576,Global Economic Monitor,,15,Global Economic Monitor,2019-09-13,NaN
